In [ ]:
trace_path = r'./dataset/20250519122059.zarr'

In [ ]:
from cracknuts.trace import ScarrTraceDataset
# 加载 scarr 格式的数据集
ds = ScarrTraceDataset.load(trace_path)

**查看曲线**

In [ ]:
# Print dataset info
ds.info()

In [ ]:
# Show trace panel
import cracknuts as cn
pt = cn.panel_trace()
pt.set_trace_dataset(ds)
pt

In [ ]:
# Change trace index.
# show_trace 接收两个索引参数，第一个参数是通道索引，第二个是曲线索引，两个参数支持高级索引
# 需要注意的是通道A和B的名称在数据里为0和1，而其索引则根曲线文件中存储的通道数量有关系，如果只有一个通道时，无论哪个通道索引都为0，如果同时有A B两个通道则A通道索引为0，B通道索引为1，可以通过 ds.channel_names 进行确认当前曲线存储的时哪个通道

# 展示第一通道的前10条曲线
# pt.show_trace[0, :10]
# 展示第一通道的后10条曲线
# pt.show_trace[0, -10:]
# 展示第一通道的 10-20 曲线
pt.show_trace[0, 10:20]
# 展示所有(两个)通道的前10条中的偶数位曲线
# pt.show_trace[:, :10:2]

**分析数据**

In [ ]:
from scarr.engines.cpa import CPA as cpa
from scarr.file_handling.trace_handler import TraceHandler as th
from scarr.model_values.sbox_weight import SboxWeight
from scarr.container.container import Container, ContainerOptions
import numpy as np

In [ ]:
handler = th(fileName=trace_path) 
model = SboxWeight() 
engine = cpa(model)
container = Container(options=ContainerOptions(engine=engine, handler=handler), model_positions = [x for x in range(16)])
container.run()

In [ ]:
candidate = np.squeeze(engine.get_candidate()) # get_candidate 获取各个字节的密钥的计算结果
' '.join(f"{x:02x}" for x in candidate) # 打印计算出的密钥

In [ ]:
result_bytes = np.squeeze(container.engine.get_result())

In [ ]:
# 打印第0个字节相关性最大的前10个相关系数机器对应的密钥猜测值
result_0_bytes = result_bytes[0] # 获取第0字节的的相关系数
row_max_indices = np.argmax(np.abs(result_0_bytes), axis=1) 
row_max_values = result_0_bytes[np.arange(result_0_bytes.shape[0]), row_max_indices]

top10_row_indices = np.argsort(np.abs(row_max_values))[::-1][:10]

for rank, row in enumerate(top10_row_indices, 1):
    col = row_max_indices[row]
    val = row_max_values[row]
    print(f"第 {rank} 候选值: 0x{row:0X}，对应的相关系数为: {val}，最高相关系数位置： {col}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_correlation_peaks(bytes_index, the_key):
    
    x = np.arange(0, 5000)
    
    fig, ax = plt.subplots(figsize=(30, 4))
    
    for i in range(256):
        if i == the_key:
            continue
        ax.plot(x, result_bytes[bytes_index, i, :5000], color='gray', linewidth=0.5, alpha=0.3)
            
    ax.plot(x, result_bytes[bytes_index, the_key, :5000], color='red', linewidth=1.0)
    
    ax.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    
    plt.show()

In [ ]:
# 把第0个字节的256个密钥猜测下的相关系数曲线画出来。正确密钥用红色画出，其它密钥用灰色画出。可以看出正确密钥对应的相关系数曲线存在最明显的尖峰。
plot_correlation_peaks(0, 0x11)

In [ ]:
# 同样，我们可以查看第1个字节的256个密钥猜测值的相关系数曲线
plot_correlation_peaks(1, 0x22)

In [ ]:
# 同样，我们可以查看第2个字节的256个密钥猜测值的相关系数曲线
plot_correlation_peaks(2, 0x33)

In [ ]:
# 同样，我们可以查看第3个字节的256个密钥猜测值的相关系数曲线
plot_correlation_peaks(3, 0x44)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0, 5000)

fig, ax = plt.subplots(figsize=(30, 4))

ax.plot(x, result_bytes[0, 0x11, :5000].T, linewidth=1.0, label='0x11')
ax.plot(x, result_bytes[1, 0x22, :5000].T, linewidth=1.0, label='0x22')
ax.plot(x, result_bytes[2, 0x33, :5000].T, linewidth=1.0, label='0x33')
ax.plot(x, result_bytes[3, 0x44, :5000].T, linewidth=1.0, label='0x44')
ax.plot(x, result_bytes[4, 0x55, :5000].T, linewidth=1.0, label='0x55')
ax.plot(x, result_bytes[5, 0x66, :5000].T, linewidth=1.0, label='0x66')
ax.plot(x, result_bytes[6, 0x77, :5000].T, linewidth=1.0, label='0x77')
ax.plot(x, result_bytes[7, 0x88, :5000].T, linewidth=1.0, label='0x88')
ax.plot(x, result_bytes[8, 0x99, :5000].T, linewidth=1.0, label='0x99')
ax.plot(x, result_bytes[9, 0x00, :5000].T, linewidth=1.0, label='0x00')
ax.plot(x, result_bytes[10, 0xaa, :5000].T, linewidth=1.0, label='0xaa')
ax.plot(x, result_bytes[11, 0xbb, :5000].T, linewidth=1.0, label='0xbb')
ax.plot(x, result_bytes[12, 0xcc, :5000].T, linewidth=1.0, label='0xcc')
ax.plot(x, result_bytes[13, 0xdd, :5000].T, linewidth=1.0, label='0xdd')
ax.plot(x, result_bytes[14, 0xee, :5000].T, linewidth=1.0, label='0xee')
ax.plot(x, result_bytes[15, 0xff, :5000].T, linewidth=1.0, label='0xff')

ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='upper right', fontsize='small', ncol=2)

plt.tight_layout()

# 显示图像
plt.show()